In [1]:
import numpy as np
import pickle as pkl

from src.models.train_emos import train_emos, train_and_test_emos
from src.visualization.pit import make_cpit_diagram_emos, make_cpit_hist_emos 
from src.visualization.brier_score import brier_skill_plot, brier_plot
from src.models.get_data import get_tensors, get_normalized_tensor
from src.models.emos import EMOS
from src.visualization.scoring_tables import make_table

2024-03-07 16:15:54.057296: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 16:15:54.083805: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 16:15:54.083837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-07 16:15:54.084591: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-07 16:15:54.088809: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 16:15:54.089345: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
neighbourhood_size = 11
parameter_names = ['wind_speed', 'press', 'kinetic', 'humid', 'geopot']
ignore = ['229', '285', '323']
train_folds = [1, 2]
train_data = get_normalized_tensor(neighbourhood_size, parameter_names, train_folds, ignore)

X_train = train_data['X']
y_train = train_data['y']
variances_train = train_data['variances']
mean_train = train_data['mean']
std_train = train_data['std']


print(X_train.shape)

(15733, 5)


In [3]:
test_fold = 3

X_test, y_test, variances_test = get_tensors(neighbourhood_size, parameter_names, test_fold, ignore)
X_test = (X_test - mean_train) / std_train

print(X_test.shape)

(7780, 5)


In [4]:
setup = {}

setup["num_features"] = len(parameter_names)
setup["feature_mean"] = mean_train
setup["feature_std"] = std_train
setup["features"] = parameter_names
setup["neighbourhood_size"] = neighbourhood_size

In [5]:
models_twcrps = {}
setup1 = setup

setup1["loss"] = "loss_twCRPS_sample"
setup1["chain_function"] = "chain_function_normal_cdf"
setup1["chain_function_mean"] = 11
setup1["chain_function_std"] = 1
setup1["samples"] = 300
setup1["optimizer"] = "Adam"
setup1["learning_rate"] = 0.01
setup1["forecast_distribution"] = "distr_trunc_normal"

twcrps_tn = EMOS(setup1)

setup1["forecast_distribution"] = "distr_log_normal"

twcrps_ln = EMOS(setup1)

setup1["forecast_distribution"] = "distr_gev"
setup1["samples"] = 300

twcrps_gev = EMOS(setup1)

setup1["forecast_distribution"] = "distr_frechet"

twcrps_frechet = EMOS(setup1)





models_twcrps["twcrps_tn"] = twcrps_tn
models_twcrps["twcrps_ln"] = twcrps_ln
models_twcrps["twcrps_gev"] = twcrps_gev
models_twcrps["twcrps_frechet"] = twcrps_frechet


print(len(models_twcrps))

Using default parameters for truncated normal distribution
Using default parameters for Log Normal distribution
Using default parameters for Generalized Extreme Value distribution
Using default parameters for Frechet distribution
4


In [6]:
epochs = 600

for model in models_twcrps:
    models_twcrps[model].fit(X_train, y_train, variances_train, epochs, printing=False)
    print("Model: ", models_twcrps[model])

Final loss:  0.08807499
Model:  EMOS Model Information:
Loss function: loss_twCRPS_sample (Samples: 300)
Forecast distribution: distr_trunc_normal
Parameters:
  a_tn: [0.9534113]
  b_tn: [ 0.8797114  -1.1838744  -0.1318484  -0.14606379  1.2007395 ]
  c_tn: [2.9175513]
  d_tn: [0.725646]
Features: wind_speed, press, kinetic, humid, geopot
Number of features: 5
Neighbourhood size: 11
Chaining function: chain_function_normal_cdf (Mean: 11.0, Std: 1.0)
Optimizer: Adam
Learning rate: 0.009999999776482582

Final loss:  0.08865459
Model:  EMOS Model Information:
Loss function: loss_twCRPS_sample (Samples: 300)
Forecast distribution: distr_log_normal
Parameters:
  a_ln: [1.6291347]
  b_ln: [ 0.06679348 -0.10326554 -0.00933671 -0.0155539   0.10231003]
  c_ln: [0.02253972]
  d_ln: [0.00232143]
Features: wind_speed, press, kinetic, humid, geopot
Number of features: 5
Neighbourhood size: 11
Chaining function: chain_function_normal_cdf (Mean: 11.0, Std: 1.0)
Optimizer: Adam
Learning rate: 0.0099999

In [7]:
setup1["forecast_distribution"] = "distr_mixture"
setup1["distribution_1"] = "distr_trunc_normal"
setup1["distribution_2"] = "distr_log_normal"

twcrps_mix_tn_ln = EMOS(setup1)
twcrps_mix_tn_ln.set_parameters(models_twcrps["twcrps_tn"].get_parameters())
twcrps_mix_tn_ln.set_parameters(models_twcrps["twcrps_ln"].get_parameters())

setup1["distribution_2"] = "distr_gev"
twcrps_mix_tn_gev = EMOS(setup1)

twcrps_mix_tn_gev.set_parameters(models_twcrps["twcrps_tn"].get_parameters())
twcrps_mix_tn_gev.set_parameters(models_twcrps["twcrps_gev"].get_parameters())

setup1["distribution_2"] = "distr_frechet"
twcrps_mix_tn_frechet = EMOS(setup1)

twcrps_mix_tn_frechet.set_parameters(models_twcrps["twcrps_tn"].get_parameters())
twcrps_mix_tn_frechet.set_parameters(models_twcrps["twcrps_frechet"].get_parameters())

models_twcrps["twcrps_mix_tn_ln"] = twcrps_mix_tn_ln
models_twcrps["twcrps_mix_tn_gev"] = twcrps_mix_tn_gev
models_twcrps["twcrps_mix_tn_frechet"] = twcrps_mix_tn_frechet

Using default parameters for truncated normal distribution
Using default parameters for Log Normal distribution
Using default weight parameter for Mixture distribution
Parameter a_tn set to [0.9534113]
Parameter b_tn set to [ 0.8797114  -1.1838744  -0.1318484  -0.14606379  1.2007395 ]
Parameter c_tn set to [2.9175513]
Parameter d_tn set to [0.725646]
Parameter a_ln set to [1.6291347]
Parameter b_ln set to [ 0.06679348 -0.10326554 -0.00933671 -0.0155539   0.10231003]
Parameter c_ln set to [0.02253972]
Parameter d_ln set to [0.00232143]
Using default parameters for truncated normal distribution
Using default parameters for Generalized Extreme Value distribution
Using default weight parameter for Mixture distribution
Parameter a_tn set to [0.9534113]
Parameter b_tn set to [ 0.8797114  -1.1838744  -0.1318484  -0.14606379  1.2007395 ]
Parameter c_tn set to [2.9175513]
Parameter d_tn set to [0.725646]
Parameter a_gev set to [1.3011674]
Parameter b_gev set to [ 0.81226045 -0.9397855  -0.13331

In [8]:
models_twcrps["twcrps_mix_tn_ln"].fit(X_train, y_train, variances_train, epochs, printing=False)
print("Model: ", models_twcrps["twcrps_mix_tn_ln"])

models_twcrps["twcrps_mix_tn_gev"].fit(X_train, y_train, variances_train, epochs, printing=False)
print("Model: ", models_twcrps["twcrps_mix_tn_gev"])

models_twcrps["twcrps_mix_tn_frechet"].fit(X_train, y_train, variances_train, epochs, printing=False)
print("Model: ", models_twcrps["twcrps_mix_tn_frechet"])

Final loss:  0.08729197
Model:  EMOS Model Information:
Loss function: loss_twCRPS_sample (Samples: 300)
Forecast distribution: distr_mixture
Distribution 1: distr_trunc_normal
Distribution 2: distr_log_normal
Mixture weight: [0.815861]
Parameters:
  weight: [0.815861]
  a_tn: [0.8410446]
  b_tn: [ 0.95003873 -0.5831397  -0.2337767   0.7219191   1.2121167 ]
  c_tn: [3.564786]
  d_tn: [0.7669719]
  a_ln: [1.6238265]
  b_ln: [ 0.04070468 -0.35114396  0.01846181 -0.2688914   0.16095191]
  c_ln: [0.13998362]
  d_ln: [-0.]
Features: wind_speed, press, kinetic, humid, geopot
Number of features: 5
Neighbourhood size: 11
Chaining function: chain_function_normal_cdf (Mean: 11.0, Std: 1.0)
Optimizer: Adam
Learning rate: 0.009999999776482582

Final loss:  0.08670479
Model:  EMOS Model Information:
Loss function: loss_twCRPS_sample (Samples: 300)
Forecast distribution: distr_mixture
Distribution 1: distr_trunc_normal
Distribution 2: distr_gev
Mixture weight: [0.4392965]
Parameters:
  weight: [0.43

In [9]:
setup1["forecast_distribution"] = "distr_mixture_linear"
setup1["distribution_1"] = "distr_trunc_normal"
setup1["distribution_2"] = "distr_log_normal"

twcrps_mixlinear_tn_ln = EMOS(setup1)
twcrps_mixlinear_tn_ln.set_parameters(models_twcrps["twcrps_tn"].get_parameters())
twcrps_mixlinear_tn_ln.set_parameters(models_twcrps["twcrps_ln"].get_parameters())

setup1["distribution_2"] = "distr_gev"
twcrps_mixlinear_tn_gev = EMOS(setup1)

twcrps_mixlinear_tn_gev.set_parameters(models_twcrps["twcrps_tn"].get_parameters())
twcrps_mixlinear_tn_gev.set_parameters(models_twcrps["twcrps_gev"].get_parameters())

setup1["distribution_2"] = "distr_frechet"
twcrps_mixlinear_tn_frechet = EMOS(setup1)

twcrps_mixlinear_tn_frechet.set_parameters(models_twcrps["twcrps_tn"].get_parameters())
twcrps_mixlinear_tn_frechet.set_parameters(models_twcrps["twcrps_frechet"].get_parameters())

models_twcrps["twcrps_mixlinear_tn_ln"] = twcrps_mixlinear_tn_ln
models_twcrps["twcrps_mixlinear_tn_gev"] = twcrps_mixlinear_tn_gev
models_twcrps["twcrps_mixlinear_tn_frechet"] = twcrps_mixlinear_tn_frechet

Using default parameters for truncated normal distribution
Using default parameters for Log Normal distribution
Using default weight parameters for weights in Mixture Linear distribution
Parameter a_tn set to [0.9534113]
Parameter b_tn set to [ 0.8797114  -1.1838744  -0.1318484  -0.14606379  1.2007395 ]
Parameter c_tn set to [2.9175513]
Parameter d_tn set to [0.725646]
Parameter a_ln set to [1.6291347]
Parameter b_ln set to [ 0.06679348 -0.10326554 -0.00933671 -0.0155539   0.10231003]
Parameter c_ln set to [0.02253972]
Parameter d_ln set to [0.00232143]
Using default parameters for truncated normal distribution
Using default parameters for Generalized Extreme Value distribution
Using default weight parameters for weights in Mixture Linear distribution
Parameter a_tn set to [0.9534113]
Parameter b_tn set to [ 0.8797114  -1.1838744  -0.1318484  -0.14606379  1.2007395 ]
Parameter c_tn set to [2.9175513]
Parameter d_tn set to [0.725646]
Parameter a_gev set to [1.3011674]
Parameter b_gev se

In [10]:
models_twcrps["twcrps_mixlinear_tn_ln"].fit(X_train, y_train, variances_train, epochs, printing=False)
print("Model: ", models_twcrps["twcrps_mixlinear_tn_ln"])

models_twcrps["twcrps_mixlinear_tn_gev"].fit(X_train, y_train, variances_train, epochs, printing=False)
print("Model: ", models_twcrps["twcrps_mixlinear_tn_gev"])

models_twcrps["twcrps_mixlinear_tn_frechet"].fit(X_train, y_train, variances_train, epochs, printing=False)
print("Model: ", models_twcrps["twcrps_mixlinear_tn_frechet"])

Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient contains NaN
Gradient c

In [11]:
parameter_dict = {}

for name, model in models_twcrps.items():
    parameter_dict[name] = model.to_dict()

# save the parameter_dict as pkl file in /net/pc200239/nobackup/users/hakvoort/models

with open('/net/pc200239/nobackup/users/hakvoort/models/emos_twcrps_mean_11_std_1.pkl', 'wb') as f:
    pkl.dump(parameter_dict, f)

print("Saved parameter dict as pkl file")

Saved parameter dict as pkl file


In [12]:
for name, model in models_twcrps.items():
    loss = model.loss_CRPS_sample_general(X_test, y_test, variances_test, 10000)
    print(name, loss.numpy())

twcrps_tn 0.9380642
twcrps_ln 2.0098855
twcrps_gev 1.048202
twcrps_frechet 1.1024867
twcrps_mix_tn_ln 1.0611806
twcrps_mix_tn_gev 1.0010288
twcrps_mix_tn_frechet 0.9838235
twcrps_mixlinear_tn_ln nan
twcrps_mixlinear_tn_gev 1.0632433
twcrps_mixlinear_tn_frechet 1.1045941
